In [1]:
import os

In [2]:
os.chdir("../")
%pwd
#%cd kidney-disease-classification

'/Users/alexeybrown/Documents/projects/Python/kidney-disease-classification'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]= "https://dagshub.com/aliakseibrown/kidney-disease-classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="aliakseibrown"
os.environ["MLFLOW_TRACKING_PASSWORD"]="435491a5b08f4198a561a9bd44fbe9d89f59fa04"

In [4]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


2024-01-04 15:15:42.227424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

OSError: No file or directory found at artifacts/training/model.h5

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [ ]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
          eval_config = EvaluationConfig(
                path_of_model="artifacts/training/model.h5",
                training_data= "artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/",
                all_params= self.params,
                mlflow_uri= "https://dagshub.com/aliakseibrown/kidney-disease-classification.mlflow",
                params_image_size= self.params.IMAGE_SIZE,
                params_batch_size= self.params.BATCH_SIZE
          )
          return eval_config

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= "validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path = Path( "scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0],
                 "accuracy": self.score[1]})
            # Model rgistry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to sue the Model Registry, which depends on the use case,
                # please refer to the doc for more inforamtion:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name= "VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
            


In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-12-31 16:20:20,548: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-31 16:20:20,552: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-31 16:20:20,554: INFO: common: created directory at: ['artifacts']]
Found 3732 images belonging to 4 classes.
75/75 [==============================] - 665s 9s/step - loss: 14.0034 - accuracy: 0.3218
[2023-12-31 16:31:26,998: INFO: common: json file saved at: scores.json]


2023/12/31 16:31:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/vq/rlgff26x52v8f_lld3d960cc0000gn/T/tmphascbdjd/model/data/model/assets
[2023-12-31 16:31:32,619: INFO: builder_impl: Assets written to: /var/folders/vq/rlgff26x52v8f_lld3d960cc0000gn/T/tmphascbdjd/model/data/model/assets]


/Users/alexeybrown/anaconda3/envs/kidney/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/12/31 16:32:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
